In [1]:
import pandas as pd
import os
import numpy as np
import cv2
from utils import load_detections, track_iou, create_similarity_matrix, associate_detections_to_tracks, update_tracks, draw_tracking_result, save_tracking_results, initialize_new_track
from KalmanFilter import KalmanFilter

###  Loading det object instances

In [2]:
det_file_path = '../data/det/det.txt'
detections_df = load_detections(det_file_path)
detections_df.head()

,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
0,1,-1,1689,385,146.620,332.710,67.567,-1,-1,-1
1,1,-1,1303,503,61.514,139.590,29.439,-1,-1,-1
2,1,-1,1258,569,40.123,91.049,19.601,-1,-1,-1
3,1,-1,31,525,113.370,257.270,17.013,-1,-1,-1
4,1,-1,1800,483,94.660,214.810,11.949,-1,-1,-1


#### Interface (main loop)

In [3]:
# Main loop
image_frames_path = '../data/img1/' 

# Define properties for the output video
fps = 40
frame_size = (1920, 1080)  # Adjust this to match the size of your frames
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec
out = cv2.VideoWriter('output_with_tracking.mp4', fourcc, fps, frame_size)
delay = int(1000/fps)

# Initialize tracking variables
tracks = []
frame_number = 1
sigma_iou = 0.2

tracking_file = 'track.txt'
if os.path.isfile(tracking_file):
    open(tracking_file, 'w').close()
else:
    open(tracking_file, 'x')

# Process each image frame
for filename in sorted(os.listdir(image_frames_path)):
    if filename.endswith(".jpg"):
        frame_path = os.path.join(image_frames_path, filename)
        frame = cv2.imread(frame_path)

        # Get current detections for this frame
        current_detections_df = detections_df[detections_df['frame'] == frame_number]
        current_boxes = current_detections_df[['bb_left', 'bb_top', 'bb_width', 'bb_height']].values
        current_confidences = current_detections_df['conf'].values

        # Predict next state for each track using the Kalman Filter
        for track in tracks:
            track['kf'].predict()

        # Compute similarity matrix and associate detections to tracks
        if frame_number > 1:
            similarity_matrix = create_similarity_matrix(current_boxes, tracks) # Similarity matrix between detections and predicted states
            matches, unmatched_detections, unmatched_tracks = associate_detections_to_tracks(similarity_matrix, sigma_iou)
            tracks = update_tracks(matches, unmatched_tracks, unmatched_detections, current_boxes, current_confidences, tracks, frame_number)
        else:
            for i, det in enumerate(current_boxes):
                tracks.append(initialize_new_track(det, current_confidences[i], frame_number))
        
        # Draw tracking
        frame_with_tracking = draw_tracking_result(frame, tracks)
        # Save the tracking results
        save_tracking_results(tracks, tracking_file, frame_number)

        # Show the frame
        cv2.imshow('Tracking', frame_with_tracking)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
        out.write(frame_with_tracking)

        frame_number += 1

out.release()
cv2.destroyAllWindows()

matches: 5, unmatched_detections: 1, unmatched_tracks: 0
matches: 5, unmatched_detections: 2, unmatched_tracks: 1
matches: 5, unmatched_detections: 1, unmatched_tracks: 2
matches: 4, unmatched_detections: 0, unmatched_tracks: 2
matches: 4, unmatched_detections: 2, unmatched_tracks: 0
matches: 5, unmatched_detections: 1, unmatched_tracks: 1
matches: 4, unmatched_detections: 1, unmatched_tracks: 2
matches: 5, unmatched_detections: 2, unmatched_tracks: 0
matches: 6, unmatched_detections: 0, unmatched_tracks: 1
matches: 5, unmatched_detections: 1, unmatched_tracks: 1
matches: 4, unmatched_detections: 1, unmatched_tracks: 2
matches: 3, unmatched_detections: 3, unmatched_tracks: 2
matches: 5, unmatched_detections: 1, unmatched_tracks: 1
matches: 5, unmatched_detections: 1, unmatched_tracks: 1
matches: 5, unmatched_detections: 1, unmatched_tracks: 1
matches: 4, unmatched_detections: 0, unmatched_tracks: 2
matches: 3, unmatched_detections: 3, unmatched_tracks: 1
matches: 6, unmatched_detection

KeyboardInterrupt: 

: 